In [0]:
%sql
create table car_bronze01
(car_name STRING, car_brand STRING, units_sold INT, car_type string)
using DELTA TBLPROPERTIES (delta.enableChangeDataFeed=true);


In [0]:
#insert the data in python notebook
data=[
    ('LC300','Toyota',5,'SUV'),
    ('Creta','Hyundai',750,'SUV'),
    ('Breeza','Suzuki',20,'CUV'),
    ('Urus','Lamborghini',75,'SUV'),
    ('Bentayga','Bentley',200,'SUV'),
    ]
columns=['car_name', 'car_brand','units_sold','car_type']
df=spark.createDataFrame(data=data,schema=columns)
df.createOrReplaceTempView("car_bronze01_table01")

In [0]:
%sql
INSERT INTO car_bronze01 TABLE car_bronze01_table01

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
select * from table_changes ("car_bronze01", 1)

car_name,car_brand,units_sold,car_type,_change_type,_commit_version,_commit_timestamp
Urus,Lamborghini,75,SUV,insert,1,2023-05-24T08:48:16.000+0000
Bentayga,Bentley,200,SUV,insert,1,2023-05-24T08:48:16.000+0000
Breeza,Suzuki,20,CUV,insert,1,2023-05-24T08:48:16.000+0000
Creta,Hyundai,750,SUV,insert,1,2023-05-24T08:48:16.000+0000
LC300,Toyota,5,SUV,insert,1,2023-05-24T08:48:16.000+0000


In [0]:
#insert the data in python notebook
data1=[
    ('Explorer','Ford',25,'SUV'),
    ('Dmax','Isuzu',75,'Pickup'),
    ('Hexa','Tata',25,'MPV'),
    ('Kona','Hyundai',125,'EV'),
    ('Charger','Dodge',20,'SUV'),
    ]
columns1=['car_name', 'car_brand','units_sold','car_type']
df1=spark.createDataFrame(data=data,schema=columns)
df1.createOrReplaceTempView("car_bronze01_table01")

In [0]:
%sql
INSERT INTO car_bronze01 TABLE car_bronze01_table01

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
select * from table_changes ("car_bronze01", 2)

car_name,car_brand,units_sold,car_type,_change_type,_commit_version,_commit_timestamp
Urus,Lamborghini,75,SUV,insert,2,2023-05-24T09:08:36.000+0000
Bentayga,Bentley,200,SUV,insert,2,2023-05-24T09:08:36.000+0000
Breeza,Suzuki,20,CUV,insert,2,2023-05-24T09:08:36.000+0000
Creta,Hyundai,750,SUV,insert,2,2023-05-24T09:08:36.000+0000
LC300,Toyota,5,SUV,insert,2,2023-05-24T09:08:36.000+0000


In [0]:
%sql
create table car_silver01
(car_name STRING, car_brand STRING, units_sold INT, car_type string)
using DELTA TBLPROPERTIES (delta.enableChangeDataFeed=true);

In [0]:
%sql
insert into car_silver01(SELECT car_name,car_brand,units_sold,car_type FROM table_changes('car_bronze01',1))

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
SELECT * FROM table_changes('car_silver01', 1)

car_name,car_brand,units_sold,car_type,_change_type,_commit_version,_commit_timestamp
Urus,Lamborghini,75,SUV,insert,1,2023-05-24T09:16:39.000+0000
Bentayga,Bentley,200,SUV,insert,1,2023-05-24T09:16:39.000+0000
Urus,Lamborghini,75,SUV,insert,1,2023-05-24T09:16:39.000+0000
Bentayga,Bentley,200,SUV,insert,1,2023-05-24T09:16:39.000+0000
Breeza,Suzuki,20,CUV,insert,1,2023-05-24T09:16:39.000+0000
Breeza,Suzuki,20,CUV,insert,1,2023-05-24T09:16:39.000+0000
Creta,Hyundai,750,SUV,insert,1,2023-05-24T09:16:39.000+0000
Creta,Hyundai,750,SUV,insert,1,2023-05-24T09:16:39.000+0000
LC300,Toyota,5,SUV,insert,1,2023-05-24T09:16:39.000+0000
LC300,Toyota,5,SUV,insert,1,2023-05-24T09:16:39.000+0000


In [0]:
%sql
create table car_gold01
(car_name STRING, car_brand STRING, units_sold INT, car_type string)
using DELTA TBLPROPERTIES(delta.enableChangeDataFeed=true);

In [0]:
%sql
INSERT into car_gold01
select car_name,car_brand,avg(units_sold) as avg_units,car_type from car_silver01 group by car_name, car_brand, car_type

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
select * from car_gold01

car_name,car_brand,units_sold,car_type
Urus,Lamborghini,75,SUV
Bentayga,Bentley,200,SUV
Breeza,Suzuki,20,CUV
Creta,Hyundai,750,SUV
LC300,Toyota,5,SUV


In [0]:
%sql
merge into car_silver01
using
(select * from table_changes ('car_bronze01',2))
as car_bronze_1
on car_bronze_1.car_name=car_silver01.car_name and
car_bronze_1.car_brand=car_silver01.car_brand and
car_bronze_1.units_sold=car_silver01.units_sold
when matched then
update set car_silver01.car_type= car_silver01.car_type
when not matched
then insert (car_name, car_brand, units_sold, car_type) values (car_name, car_brand, units_sold, car_type)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,10,0,0


In [0]:
%sql
select * from table_changes ("car_silver01",1)

car_name,car_brand,units_sold,car_type,_change_type,_commit_version,_commit_timestamp
Urus,Lamborghini,75,SUV,update_preimage,2,2023-05-24T11:12:46.000+0000
Urus,Lamborghini,75,SUV,update_postimage,2,2023-05-24T11:12:46.000+0000
Urus,Lamborghini,75,SUV,update_preimage,2,2023-05-24T11:12:46.000+0000
Urus,Lamborghini,75,SUV,update_postimage,2,2023-05-24T11:12:46.000+0000
Bentayga,Bentley,200,SUV,update_preimage,2,2023-05-24T11:12:46.000+0000
Bentayga,Bentley,200,SUV,update_postimage,2,2023-05-24T11:12:46.000+0000
Bentayga,Bentley,200,SUV,update_preimage,2,2023-05-24T11:12:46.000+0000
Bentayga,Bentley,200,SUV,update_postimage,2,2023-05-24T11:12:46.000+0000
Breeza,Suzuki,20,CUV,update_preimage,2,2023-05-24T11:12:46.000+0000
Breeza,Suzuki,20,CUV,update_postimage,2,2023-05-24T11:12:46.000+0000
